In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append('./dataset')
from dataset import *

sys.path.append('./pytorch_pipeline')
from ptpl import PyTorchPipeline

sys.path.append('./model/')
from model import *

import torch
import torch.nn as nn

from torchdyn.models import *
# from torchdyn.datasets import *
from torchdyn import *

import random
from tqdm import tqdm


In [3]:
torch.set_num_threads(8)

In [4]:
hidden_dim = 64
emb_dim = 32


hparams = {
    'wb' : False,
    'hidden_dim': hidden_dim,
    'emb_dim': emb_dim,
    'num_epochs': 100,
    'num_batches' : 32,
    'path2save': "./weights/hidden_size_" + str(hidden_dim) + "_emb_dim_" + str(emb_dim) + ".pt",
    'learning_rate': 1e-3,
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# prepare data

path = os.path.join("./data", "eng-fra.txt")
perSentence, vocab = get_loaders(path)
vocab_size = len(vocab)

trainPerSentence = {}
valPerSentence = {}

for key, values in perSentence.items():
    arr = values[:]
    random.shuffle(arr)

    val_size = int(0.2 * len(arr))

    val_arr = arr[:val_size]
    train_arr = arr[val_size:]

    trainPerSentence[key] = train_arr[:]
    valPerSentence[key] = val_arr[:]

del val_arr, train_arr, perSentence
train_size = sum([len(value) for value in trainPerSentence.values() ])
val_size = sum([len(value) for value in valPerSentence.values() ])


print("train_size: ", train_size)
print("val_size: ", val_size)

Total number of sentences is 135842 



100%|██████████| 135842/135842 [00:00<00:00, 501270.54it/s]


The vocabulary size if 5384 

The size of the newly created text data is 122885, 90% of the original text
train_size:  98310
val_size:  24575


In [6]:
# define a model

embedding = nn.Embedding(vocab_size, emb_dim)
encoder = Encoder(emb_dim, hidden_dim)
decoder = Decoder(hidden_dim, vocab_size)

model = Seq2Seq(embedding, encoder, decoder)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [8]:
ptpl = PyTorchPipeline(
    project_name = "gru_node",
    configs = {
        'device': device,
        'criterion': criterion,
        'optimizer': optimizer,
        'train_dataloader': trainPerSentence,
        'val_dataloader': valPerSentence,
        'print_logs': True,
        'wb': hparams['wb'],
    },
    hparams = hparams,
    model = model,
)

PyTorch pipeline for gru_node is set up


In [9]:
ptpl.train(num_epochs= 1)

The path to save the model is not provided. Thus the model weigths will not be saved.


100%|██████████| 3076/3076 [02:05<00:00, 24.44it/s]


Epoch: 1/1 || Loss: 5.377899169921875


100%|██████████| 772/772 [00:13<00:00, 59.02it/s]

Validation loss is 0.1660517007112503


In [ ]:
print(torch.get_num_threads())